In [1]:
import joblib
import os

# 指定模型所在的目录和文件名
model_directory = 'saved_models'
model_filename = 'random_forest_model.joblib'

# 完整的模型加载路径
model_path = os.path.join(model_directory, model_filename)

# 加载随机森林模型
loaded_rf = joblib.load(model_path)

print("随机森林模型已成功加载")

随机森林模型已成功加载


In [2]:
# 导入必要的库
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# 读取CSV文件
df = pd.read_csv('cleaned_data/cleaned_charging_station_data.csv')
df = df.reset_index(drop=True)
# 显示数据的基本信息


df = df.drop(['is_weekend', 'time_of_day', 'is_holiday','is_work_hour','connector_unique_id','usage_last_24h','usage_last_7d','city_density_level','availability_24h_ago','availability_1week_ago','availability_change','relative_days'], axis=1)

from sklearn.preprocessing import LabelEncoder
df['station_name'] = df['station_name'].astype(str)
le = LabelEncoder()
df['station_name_encoded'] = le.fit_transform(df['station_name'])
df = df.drop('station_name', axis=1)
df['city_id_encoded'] = le.fit_transform(df['city_id'])
df = df.drop('city_id', axis=1)

X = df.drop('is_available', axis=1)
y = df['is_available']

# 创建周期性特征
X['hour_sin'] = np.sin(X['hour'] * (2 * np.pi / 24))
X['hour_cos'] = np.cos(X['hour'] * (2 * np.pi / 24))
X['day_of_week_sin'] = np.sin(X['day_of_week'] * (2 * np.pi / 7))
X['day_of_week_cos'] = np.cos(X['day_of_week'] * (2 * np.pi / 7))
X = X.drop('hour', axis=1)
X = X.drop('day_of_week', axis=1)


# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1067/986144611.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_data/cleaned_charging_station_data.csv')


In [4]:


# 在测试集上进行预测
y_pred = loaded_rf.predict(X_test)
y_pred_proba = loaded_rf.predict_proba(X_test)[:, 1]

# 打印分类报告
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 打印混淆矩阵
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# 计算 AUC-ROC 分数
auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC-ROC Score: {auc_roc:.4f}")

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.69      0.72    157087
           1       0.89      0.92      0.91    441182

    accuracy                           0.86    598269
   macro avg       0.82      0.80      0.81    598269
weighted avg       0.86      0.86      0.86    598269


Confusion Matrix:
[[108304  48783]
 [ 36121 405061]]

AUC-ROC Score: 0.9171
